In [24]:
import tensorflow as tf
import pandas as pd
from IPython.display import clear_output

In [42]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [43]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train_data = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test_data = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [44]:
train_data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [45]:
train_data_species = train_data.pop('Species')
test_data_species = test_data.pop('Species')
train_data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [46]:
def input_func(data_df, label_df, training=True, batch_size=256):
  # convert the input dataframe into a dataset
  ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))

  if training:
    ds = ds.shuffle(1000).repeat()

  ds = ds.batch(batch_size)
  return ds

In [47]:
train_data.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [48]:
feature_cols = []

for key in train_data.keys():
  feature_cols.append(tf.feature_column.numeric_column(key=key))

In [49]:
feature_cols

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [50]:
# DNN classifier with two hidden layers of 30 and 10 nodes respectively.
# number of classes is 3 as there are 3 classes t ochoose from
dnn_classifier_func = tf.estimator.DNNClassifier(hidden_units=[30, 10], feature_columns=feature_cols, n_classes=3)

RuntimeError: ignored

In [51]:
dnn_classifier_func.train(input_fn=lambda: input_func(data_df=train_data, label_df=train_data_species), steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqb06tpzq/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpqb06tpzq/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.3361923, step = 5000
INFO:tensorflow:global_step/sec: 430.343
INFO:tensorflow:loss = 0.3304839, step = 5100 (0.235 sec)
INFO:tensorflow:global_step/sec: 533.288
INFO:tensorflow:loss = 0.31981248, step = 5200 (0.189 sec)
INFO:tensorflow:global_step/sec: 505.051
INFO:tensorflow:loss = 0.3230422, step = 5300 (0.198 sec)
INFO:tensorflow:global_step/sec: 455.119


In [52]:
evaluation_result = dnn_classifier_func.evaluate(input_fn=lambda: input_func(data_df=test_data, label_df=test_data_species, training=False))
evaluation_result

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-02T18:43:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpqb06tpzq/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.21700s
INFO:tensorflow:Finished evaluation at 2021-05-02-18:43:03
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.96666664, average_loss = 0.24172701, global_step = 10000, loss = 0.24172701
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmpqb06tpzq/model.ckpt-10000


{'accuracy': 0.96666664,
 'average_loss': 0.24172701,
 'global_step': 10000,
 'loss': 0.24172701}

In [53]:
def input_func_for_prediction(data_df, batch_size=256):
  ds = tf.data.Dataset.from_tensor_slices(dict(data_df)).batch(batch_size)
  
  return ds

In [60]:
data_for_prediction = {}

print("Enter numeric values for making predictions: ")
for key in train_data.keys():
  valid = True
  while valid: 
    input_value = input(key + ": ")
    if not input_value.isdigit(): valid = False

  data_for_prediction[key] = [float(input_value)]

predictions = dnn_classifier_func.predict(input_fn=lambda:input_func_for_prediction(data_df=data_for_prediction))

for prediction in predictions:
  class_id = prediction['class_ids'][0]
  iris_species = SPECIES[class_id]
  probability = round(prediction['probabilities'][class_id] * 100, 2)

  clear_output()
  print('Flower species: "' + iris_species + '" , Probability: ' + str(probability) + "%")

Flower species: "Setosa" , Probability: 96.15%
